In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv
/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv
/kaggle/input/llm-detect-ai-generated-text/test_essays.csv
/kaggle/input/llm-detect-ai-generated-text/train_essays.csv


In [2]:
import pandas as pd

In [3]:
train_eassy= pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/train_essays.csv")

In [4]:
train_eassy

,id,prompt_id,text,generated
0,0059830c,0,Cars. Cars have been around since they became ...,0
1,005db917,0,Transportation is a large necessity in most co...,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0
3,00940276,0,How often do you ride in a car? Do you drive a...,0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0
...,...,...,...,...
1373,fe6ff9a5,1,There has been a fuss about the Elector Colleg...,0
1374,ff669174,0,Limiting car usage has many advantages. Such a...,0
1375,ffa247e0,0,There's a new trend that has been developing f...,0
1376,ffc237e9,0,As we all know cars are a big part of our soci...,0


In [5]:
train_prompts = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv")

In [6]:
train_prompts

,prompt_id,prompt_name,instructions,source_text
0,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
1,1,Does the electoral college work?,Write a letter to your state senator in which ...,# What Is the Electoral College? by the Office...


In [7]:
df= pd.merge(train_eassy, train_prompts, on='prompt_id')

In [8]:
df

,id,prompt_id,text,generated,prompt_name,instructions,source_text
0,0059830c,0,Cars. Cars have been around since they became ...,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
1,005db917,0,Transportation is a large necessity in most co...,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
3,00940276,0,How often do you ride in a car? Do you drive a...,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
...,...,...,...,...,...,...,...
1373,fe6ff9a5,1,There has been a fuss about the Elector Colleg...,0,Does the electoral college work?,Write a letter to your state senator in which ...,# What Is the Electoral College? by the Office...
1374,ff669174,0,Limiting car usage has many advantages. Such a...,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
1375,ffa247e0,0,There's a new trend that has been developing f...,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
1376,ffc237e9,0,As we all know cars are a big part of our soci...,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."


In [9]:
df.isnull().sum()

id              0
prompt_id       0
text            0
generated       0
prompt_name     0
instructions    0
source_text     0
dtype: int64

In [10]:
df= df.drop(["id"] ,axis=1)
# droped prompt_id and id

In [11]:
x =df.drop(["generated"],axis=1)

In [12]:
y= df["generated"]

In [13]:
y

0       0
1       0
2       0
3       0
4       0
       ..
1373    0
1374    0
1375    0
1376    0
1377    0
Name: generated, Length: 1378, dtype: int64

In [14]:
x

,prompt_id,text,prompt_name,instructions,source_text
0,0,Cars. Cars have been around since they became ...,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
1,0,Transportation is a large necessity in most co...,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
2,0,"""America's love affair with it's vehicles seem...",Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
3,0,How often do you ride in a car? Do you drive a...,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
4,0,Cars are a wonderful thing. They are perhaps o...,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
...,...,...,...,...,...
1373,1,There has been a fuss about the Elector Colleg...,Does the electoral college work?,Write a letter to your state senator in which ...,# What Is the Electoral College? by the Office...
1374,0,Limiting car usage has many advantages. Such a...,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
1375,0,There's a new trend that has been developing f...,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
1376,0,As we all know cars are a big part of our soci...,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."


In [15]:
x["prompt_name"].unique()

array(['Car-free cities', 'Does the electoral college work?'],
      dtype=object)

In [16]:
x

,prompt_id,text,prompt_name,instructions,source_text
0,0,Cars. Cars have been around since they became ...,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
1,0,Transportation is a large necessity in most co...,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
2,0,"""America's love affair with it's vehicles seem...",Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
3,0,How often do you ride in a car? Do you drive a...,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
4,0,Cars are a wonderful thing. They are perhaps o...,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
...,...,...,...,...,...
1373,1,There has been a fuss about the Elector Colleg...,Does the electoral college work?,Write a letter to your state senator in which ...,# What Is the Electoral College? by the Office...
1374,0,Limiting car usage has many advantages. Such a...,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
1375,0,There's a new trend that has been developing f...,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
1376,0,As we all know cars are a big part of our soci...,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."


In [17]:
from sklearn.preprocessing import LabelEncoder

In [18]:
columns_to_encode = ['prompt_name', 'instructions','source_text']

le = LabelEncoder()

for column in columns_to_encode:
    x[column] = le.fit_transform(x[column])


In [19]:
df

,prompt_id,text,generated,prompt_name,instructions,source_text
0,0,Cars. Cars have been around since they became ...,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
1,0,Transportation is a large necessity in most co...,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
2,0,"""America's love affair with it's vehicles seem...",0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
3,0,How often do you ride in a car? Do you drive a...,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
4,0,Cars are a wonderful thing. They are perhaps o...,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
...,...,...,...,...,...,...
1373,1,There has been a fuss about the Elector Colleg...,0,Does the electoral college work?,Write a letter to your state senator in which ...,# What Is the Electoral College? by the Office...
1374,0,Limiting car usage has many advantages. Such a...,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
1375,0,There's a new trend that has been developing f...,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
1376,0,As we all know cars are a big part of our soci...,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."


In [20]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [22]:
stop_words = set(stopwords.words('english'))

In [23]:
def remove_stopwords(text):
    words = word_tokenize(text)  
    words_filtered = [word for word in words if word.lower() not in stop_words]
    return ' '.join(words_filtered)

In [24]:
x['text'] = x['text'].apply(remove_stopwords)

In [25]:
x

,prompt_id,text,prompt_name,instructions,source_text
0,0,"Cars . Cars around since became famous 1900s ,...",0,1,0
1,0,Transportation large necessity countries world...,0,1,0
2,0,`` America 's love affair 's vehicles seems co...,0,1,0
3,0,often ride car ? drive one motor vehicle work ...,0,1,0
4,0,Cars wonderful thing . perhaps one worlds grea...,0,1,0
...,...,...,...,...,...
1373,1,fuss Elector College . Many people get confuse...,1,0,1
1374,0,Limiting car usage many advantages . putting l...,0,1,0
1375,0,"'s new trend developing years , soon full thro...",0,1,0
1376,0,know cars big part society today . However car...,0,1,0


In [26]:
from nltk.tokenize import word_tokenize

In [27]:
import string

In [28]:
custom_punctuation = string.punctuation + '‘’“”'

In [29]:
def remove_punctuation(text):
    words = word_tokenize(text)
    words_filtered = [word for word in words if word not in custom_punctuation]
    return ' '.join(words_filtered)

In [30]:
import string

In [31]:
x['text'] = x['text'].apply(remove_punctuation)

In [32]:
x['text'][0]

"Cars Cars around since became famous 1900s Henry Ford created built first ModelT Cars played major role every day lives since people starting question limiting car usage would good thing limiting use cars might good thing like matter article `` German Suburb Life Goes Without Cars '' Elizabeth Rosenthal states automobiles linchpin suburbs middle class families either Shanghai Chicago tend make homes Experts say huge impediment current efforts reduce greenhouse gas emissions tailpipe Passenger cars responsible 12 percent greenhouse gas emissions Europe ... 50 percent carintensive areas United States Cars main reason greenhouse gas emissions lot people driving around time getting need go Article `` Paris bans driving due smog '' Robert Duffer says Paris days nearrecord pollution enforced partial driving ban clear air global city also says Monday motorist evennumbered license plates ordered leave cars home fined 22euro fine 31 order would applied oddnumbered plates following day Cars rea

In [33]:
def remove_quotes(text):
    
    quotes_to_remove = ['"', "'", '‘', '’', '“', '”',"''",'""',"``","."]
    
    for quote in quotes_to_remove:
        text = text.replace(quote, "")
    return text

In [34]:
x['text'] = x['text'].apply(remove_quotes)

In [35]:
x['text'][0]

'Cars Cars around since became famous 1900s Henry Ford created built first ModelT Cars played major role every day lives since people starting question limiting car usage would good thing limiting use cars might good thing like matter article  German Suburb Life Goes Without Cars  Elizabeth Rosenthal states automobiles linchpin suburbs middle class families either Shanghai Chicago tend make homes Experts say huge impediment current efforts reduce greenhouse gas emissions tailpipe Passenger cars responsible 12 percent greenhouse gas emissions Europe  50 percent carintensive areas United States Cars main reason greenhouse gas emissions lot people driving around time getting need go Article  Paris bans driving due smog  Robert Duffer says Paris days nearrecord pollution enforced partial driving ban clear air global city also says Monday motorist evennumbered license plates ordered leave cars home fined 22euro fine 31 order would applied oddnumbered plates following day Cars reason polluti

In [36]:
def convert_to_lowercase(text):
    return text.lower()

In [37]:
x['text'] = x['text'].apply(convert_to_lowercase)

In [38]:
x['text'][0]

'cars cars around since became famous 1900s henry ford created built first modelt cars played major role every day lives since people starting question limiting car usage would good thing limiting use cars might good thing like matter article  german suburb life goes without cars  elizabeth rosenthal states automobiles linchpin suburbs middle class families either shanghai chicago tend make homes experts say huge impediment current efforts reduce greenhouse gas emissions tailpipe passenger cars responsible 12 percent greenhouse gas emissions europe  50 percent carintensive areas united states cars main reason greenhouse gas emissions lot people driving around time getting need go article  paris bans driving due smog  robert duffer says paris days nearrecord pollution enforced partial driving ban clear air global city also says monday motorist evennumbered license plates ordered leave cars home fined 22euro fine 31 order would applied oddnumbered plates following day cars reason polluti

In [39]:
from nltk.stem.porter import PorterStemmer

In [40]:
ps=PorterStemmer()

In [41]:
def stem(text):
    y=[]
    
    for i in text.split():
        y.append(ps.stem(i))
        
    return ' '.join(y)
        

In [42]:
x['text'] = x['text'].apply(stem)

In [43]:
x['text'][0]

'car car around sinc becam famou 1900 henri ford creat built first modelt car play major role everi day live sinc peopl start question limit car usag would good thing limit use car might good thing like matter articl german suburb life goe without car elizabeth rosenth state automobil linchpin suburb middl class famili either shanghai chicago tend make home expert say huge impedi current effort reduc greenhous ga emiss tailpip passeng car respons 12 percent greenhous ga emiss europ 50 percent carintens area unit state car main reason greenhous ga emiss lot peopl drive around time get need go articl pari ban drive due smog robert duffer say pari day nearrecord pollut enforc partial drive ban clear air global citi also say monday motorist evennumb licens plate order leav car home fine 22euro fine 31 order would appli oddnumb plate follow day car reason pollut entir citi like pari show bad car pollut caus entir citi likewis articl carfre day spin big hit bogota andrew selski say program s

In [44]:
x['text'].tail()

1373    fuss elector colleg mani peopl get confus work...
1374    limit car usag mani advantag put lot less poll...
1375    s new trend develop year soon full throttl aff...
1376    know car big part societi today howev car bigg...
1377    car around sinc 1800 s popular ever sinc altho...
Name: text, dtype: object

In [45]:
from sklearn.feature_extraction.text import TfidfTransformer

In [46]:
x

,prompt_id,text,prompt_name,instructions,source_text
0,0,car car around sinc becam famou 1900 henri for...,0,1,0
1,0,transport larg necess countri worldwid doubt c...,0,1,0
2,0,america s love affair s vehicl seem cool say e...,0,1,0
3,0,often ride car drive one motor vehicl work sto...,0,1,0
4,0,car wonder thing perhap one world greatest adv...,0,1,0
...,...,...,...,...,...
1373,1,fuss elector colleg mani peopl get confus work...,1,0,1
1374,0,limit car usag mani advantag put lot less poll...,0,1,0
1375,0,s new trend develop year soon full throttl aff...,0,1,0
1376,0,know car big part societi today howev car bigg...,0,1,0


In [47]:
y[704]

1

In [48]:
x.loc[704]

prompt_id                                                       1
text            essay analyz discuss prove one reason favor ke...
prompt_name                                                     1
instructions                                                    0
source_text                                                     1
Name: 704, dtype: object

In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [50]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(2,7), max_features=10000, stop_words='english', max_df=0.95)


In [51]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(2, 7), max_features=10000, stop_words='english', max_df=0.95)

# Apply TF-IDF on the 'text' column
tfidf_matrix = tfidf_vectorizer.fit_transform(x['text'])

# Convert the sparse TF-IDF matrix to a dense DataFrame
tfidf_x= pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Reset index of the original DataFrame to align with tfidf_df
x= x.reset_index(drop=True)

# Concatenate the TF-IDF features with the other columns
x = pd.concat([x[['prompt_id', 'prompt_name', 'instructions', 'source_text']], tfidf_x], axis=1)

In [52]:
x

,prompt_id,prompt_name,instructions,source_text,000 dollar,000 driver,000 driver fine,000 driver fine accord,000 driver fine accord reuter,000 home,...,york new bikeshar program skyrocket,york time,young adult,young peopl,young peopl decreas,young peopl decreas 23,young peopl decreas 23 percent,young peopl decreas 23 percent 2001,young peopl decreas 23 percent 2001 2009,young peopl drive
0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1373,1,1,0,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1374,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1375,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1376,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
y

0       0
1       0
2       0
3       0
4       0
       ..
1373    0
1374    0
1375    0
1376    0
1377    0
Name: generated, Length: 1378, dtype: int64

In [54]:
from sklearn.model_selection import train_test_split

In [55]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.4, random_state=2
                                                )

In [56]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((826, 10004), (552, 10004), (826,), (552,))

In [57]:
from sklearn.svm import SVC

In [58]:
svc= SVC(kernel="linear")

In [59]:
svc.fit(x_train,y_train)

SVC(kernel='linear')

In [60]:
y_pred= svc.predict(x_test)

In [61]:
from sklearn.metrics import accuracy_score, classification_report

In [62]:
accuracy = accuracy_score(y_test, y_pred)

In [63]:
accuracy

0.9981884057971014

In [64]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       551
           1       0.00      0.00      0.00         1

    accuracy                           1.00       552
   macro avg       0.50      0.50      0.50       552
weighted avg       1.00      1.00      1.00       552



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
